# Eerste hulp bij snelle oefeningen

## Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

## Special

In [ ]:
%matplotlib inline
%matplotlib nbagg #interactieve plot
#als nbagg ni werkt widget
%%capture

## Plot

In [ ]:
fig,ax = plt.subplots()

## meerdere plots

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2,figsize=(12,4)) #(1 rij, 2 kolommen met totale grootte figuur figsize)

## 3D plots

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
fig = plt.figure()
ax = fig.gca(projection='3d')
ax.plot(x, y, z,color='g')
plt.show()

## Vectorplot i.e meshgrid

In [ ]:
T,S = np.meshgrid(np.linspace(0,100,15),np.linspace(0,100,15))
T_ = 0.5*T - 0.01*S*T
S_ = 0.005*S*T - 0.2*S
M = np.sqrt(T_**2 + S_**2)
M[ M == 0] = 1 #vermijden dat er door 0 gedeeld wordt
T_ /= M
S_ /= M
plt.quiver(T,S,T_,S_,M)
plt.title('tuna shark')
plt.xlabel('tuna')
plt.ylabel('shark')
plt.colorbar()

## Euler

In [ ]:
#versie 1 (intiutiefst):
def euler(X_init,Y_init,Z_init,stap=0.1,stappen):
    x,y,z = [],[],[]
    Xaccent = X_init
    Yaccent = Y_init
    Zaccent = Z_init
    for _ in range(0,stappen):
        x.append(Xaccent)
        y.append(Yaccent)
        z.append(Zaccent)
        Xaccent += stap*vergelijkingen(x[-1],y[-1],z[-1])[0]
        Yaccent += stap*vergelijkingen(x[-1],y[-1],z[-1])[1]
        Zaccent += stap*vergelijkingen(x[-1],y[-1],z[-1])[2]
    return x,y,z
#versie 2:
def euler(X_init,V_init,stappen,stapgrootte):
    X = np.zeros(stappen)
    V = np.zeros(stappen)
    X[0] = X_init
    V[0] = V_init
    for i in range(stappen-1):
        dX,dV = rayleigh(X[i],V[i])
        X[i + 1] = X[i] + stapgrootte*dX
        V[i + 1] = V[i] + stapgrootte*dV
    return X,V

## Stabiliteit

In [1]:
#versie 0: manueel X' oplossen en kijken of positief/negatief of manueel eigenwaarden vinden

#versie 1: perturbatie
def stabiel(X_init,k,r):
    perturbatie = 0.01
    tijdsinterval = 0.01
    x = []
    X = X_init + perturbatie
    for i in range(100):
        X += tijdsinterval*verandering(X,k,r)
    return abs(X-X_init) <= perturbatie

#alternatief voor stabiliteitsanalyse:
def stabiel(x, f, u, perturbatie=0.01): #Bekijk de stabiliteit voor bepaalde x-waarden, met f de functies
    if f(x + perturbatie, u) < 0 and f(x - perturbatie, u) > 0:
        return (1,1) #stabiel links en rechts
    elif f(x + perturbatie, u) > 0 and f(x - perturbatie, u) < 0:
        return (0,0) #instabiel links en rechts
    elif f(x + perturbatie, u) > 0 and f(x - perturbatie, u) > 0:
        return (1,0) #stabiel links
    elif f(x + perturbatie, u) < 0 and f(x - perturbatie, u) < 0:
        return (0,1) #stabiel rechts

#### eigenwaarden-methode:

Jacobiaan: $$
\left[
  \begin{array}{cc}
    \frac{\partial X'}{\partial X} & \frac{\partial X'}{\partial V}\\
    \frac{\partial V'}{\partial X} & \frac{\partial V'}{\partial V}\\
  \end{array}
\right]$$

In [ ]:
from sympy import *

In [ ]:
#1 vergelijking:
#X = Symbol('X',real=True) skip imaginaire bois, dit mag blijkbaar niet en beter complexe ook bekijken die heel kleine imaginaire waarde hebben
X = Symbol('X')
sol = solve(R[r]*(1-X/K[k]) - X/(1+X**2),X)

Als je vergelijking simpel is:

In [ ]:
import scipy.linalg as la
M = np.array([[9/7,-4/7],[8/7,-9/7]]) #[rij],[rij2]
eigenwaarden,eigenvectoren = la.eig(M)

Anders:

In [ ]:
N,P = symbols('N P')
N_ = N*(1 - N/7) - 0.3*(N*P)/(1 + N)
P_ = 0.1*P*(1 - P/N)
oplossingen = solve([N_, P_], [N, P]) #da way om 2 vgl op te lossen
jac = Matrix([N_, P_]).jacobian([N, P]) #da way om jac uit te rekenen
jac_oplossing = jac.subs(N,oplossingen[1][0]).subs(P,oplossingen[1][1])
eigenwaarden = jac_oplossing.eigenvals()

def stabiliteit2D(eigenwaarden):
    eig1, eig2 = tuple(eigenwaarden)
    stabiliteit = np.zeros(5) #entry's met 1 corresponderen met: zie beneden
    if im(eig1) < 10**-10 and im(eig2) < 10**-10: #zo goed als reeël
        if eig1 < 0 and eig2 < 0:
            stab = "Stable node"
            stabiliteit[0] = 1
        elif eig1 > 0 and eig2 > 0:
            stab = "Unstable node"
            stabiliteit[1] = 1
        else:
            stab = "Saddle point"
            stabiliteit[2] = 1
    elif re(eig1) < 0:
        stab = "Stable spiral"
        stabiliteit[3] = 1
    else:
        stab = "Unstable spiral or limit cycle"
        stabiliteit[4] = 1
    return stab, stabiliteit

i.e negatief reëel impl. stabiel <-> onstabiel en als imaginair en reëel deel negatief: stabiele spiraal, anders onstabiele of limit cycle. 

## Animatie

### imports

In [ ]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

In [ ]:
%%capture
fig, ax = plt.subplots()
def animate(i):
    ax.clear()
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_xlim(-1,1)
    ax.set_ylim(-1,1)
    ax.plot(x,-x-a_lijst[i]*x**3+x)
    ax.set_title('a={}'.format(a_lijst[i]))

anim = FuncAnimation(fig, animate,frames=20, interval=100)

In [ ]:
HTML(anim.to_jshtml())

# Widgets

In [ ]:
%%capture
from ipywidgets import *
def vergelijking(X,r):
    return r*X*(1 - X)

#plt:
def cobweb(stappen,X_init,r):
    X = []
    Y = []
    plt.figure(figsize=(13,10))
    plt.xlim(0,1)
    plt.ylim(0,1.1)
    T_ = np.arange(0,1,0.001)
    X_ = vergelijking(T_,r)
    plt.plot(T_,X_,color = 'black')
    plt.plot(T_,T_,ls=('dotted'),lw='2',color = 'grey')
    #begin:
    X.append(X_init)
    Y.append(0)
    for stap in range(stappen):
        Y.append(vergelijking(X[-1],r))
        X.append(X[-1])
        plt.plot(X[-2:],Y[-2:],color='b')
        X.append(Y[-1])
        Y.append(Y[-1])
        plt.plot(X[-2:],Y[-2:],color='b')

#ax:
def FHN(I_ext,start,stop,stapgrootte):
    fig,ax = plt.subplots(1,figsize=(12,6))
    ax.clear()
    V,w,I = euler(0,0,I_ext,start,stop,4,stapgrootte)
    T = np.arange(0,4,stapgrootte)
    ax.plot(T,V,label='spanning')
    ax.plot(T,I,label='stroom')
    ax.legend()
    ax.set_xlim(0,3.5)
    ax.set_ylim(-1,1)
    

In [ ]:
widgets.interact(cobweb,
                 stappen=widgets.IntSlider(description="$n_{\\rm max}$", min=2, max=50, value=10, continuous_update=False),
                 X_init=widgets.FloatSlider(description="$X_0$", min=0, max=1, step=0.01, value=0.01, continuous_update=False),
                 r=widgets.FloatSlider(description="$r$", min=0, max=4, step=0.01, value=4, continuous_update=False));

i.e beetje als animatie opm: value is degene waar die op begint